In [1]:
!pip install youtube-transcript-api
!pip install gradio
import gradio as gr
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration

# Helper functions as defined before
def get_video_id(url):
    if "youtu.be" in url:
        return url.split("/")[-1]
    elif "youtube.com" in url:
        return url.split("v=")[1].split("&")[0]
    else:
        return None

def get_transcript(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        transcript = " ".join([d['text'] for d in transcript_list])
        return transcript
    except Exception as e:
        return f"Error retrieving transcript: {e}"

def summarize_transcript(transcript):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

    chunk_size = 1000
    transcript_chunks = [transcript[i:i + chunk_size] for i in range(0, len(transcript), chunk_size)]

    summaries = []
    for chunk in transcript_chunks:
        summary = summarizer(chunk, max_length=200, min_length=60, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    full_summary = " ".join(summaries)
    return full_summary

def answer_questions(full_context, question):
    qa_model = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
    result = qa_model(question=question, context=full_context)
    return result['answer']

# Main chatbot logic
def process_video_url(url):
    video_id = get_video_id(url)
    if not video_id:
        return "Invalid YouTube URL.", None

    transcript = get_transcript(video_id)
    if "Error" in transcript:
        return transcript, None

    summary = summarize_transcript(transcript)
    return summary, summary + "\n\n" + transcript

def chatbot_ui(url, question):
    summary, full_context = process_video_url(url)
    if not full_context:
        return summary, ""

    if question.strip():
        answer = answer_questions(full_context, question)
    else:
        answer = "Please ask a question about the video content."

    return summary, answer

# Setting up Gradio interface
with gr.Blocks() as ui:
    gr.Markdown("# YouTube Video Summary & Q&A Chatbot")

    with gr.Row():
        url_input = gr.Textbox(label="YouTube Video URL", placeholder="Enter a YouTube video URL")

    summary_output = gr.Textbox(label="Video Summary", placeholder="Summary of the video will appear here.")
    question_input = gr.Textbox(label="Ask a Question", placeholder="Ask any question about the video content")
    answer_output = gr.Textbox(label="Answer", placeholder="Answer will appear here.")

    # Define interactive functions
    summary_btn = gr.Button("Generate Summary")
    question_btn = gr.Button("Get Answer")

    summary_btn.click(process_video_url, inputs=url_input, outputs=[summary_output, answer_output])
    question_btn.click(chatbot_ui, inputs=[url_input, question_input], outputs=[summary_output, answer_output])

# Launch the interface
ui.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To 